In [17]:
from pprint import pprint
import requests
from bs4 import BeautifulSoup


url_list = [
    'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=1',
    'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=2' 
]

result = requests.get(url_list[0])

if result.status_code == 200:
    soup = BeautifulSoup(result.content, 'html.parser')

products = soup.select('div.items__item.item-card')
# pprint(products)


items= []

for adv in products:
    data = {
    'name' : adv.select('h2[class*=ItemTitle-module_item-title__]'),
    'price' : adv.select('p.price'),
    'town' : adv.select('span[class*=index-module_town__]'),
    'city' : adv.select('span.city'),
    'shipping' : adv.select('span.shipping-badge'), 
    'sold' : adv.select('span.item-sold-badge'),
    }

    for key, value in data.items():
        if value:
            data[key] = value[0].get_text()
        else: 
            data[key] = None
        
        # oppure si può anche scrivere così
        #data[key] = value[0].get_text() if value else None

    data.update({
        'price':float(data['price'].replace('Spedizione disponibile', '')\
                .replace('\xa0€', ' ')\
                .replace('Venduto', ''))
    })

        

    pprint(data) 
# [0].get_text().replace('Spedizione disponibile', '').replace('Venduto', '')

{'city': '(PV)',
 'name': 'Zx spectrum+2 128k sinclair',
 'price': 300.0,
 'shipping': 'Spedizione disponibile',
 'sold': None,
 'town': 'Pavia '}
{'city': '(VE)',
 'name': 'ZX Spectrum SINCLAIR',
 'price': 220.0,
 'shipping': 'Spedizione disponibile',
 'sold': None,
 'town': 'Noventa di Piave '}
{'city': '(TO)',
 'name': 'Sinclair ZX Spectrum 16kb',
 'price': 550.0,
 'shipping': 'Spedizione disponibile',
 'sold': None,
 'town': 'Torino '}
{'city': '(SA)',
 'name': 'Disco 2tb Batocera v36',
 'price': 100.0,
 'shipping': 'Spedizione disponibile',
 'sold': None,
 'town': 'Pagani '}
{'city': '(SA)',
 'name': 'Disco 2tb Batocera',
 'price': 100.0,
 'shipping': 'Spedizione disponibile',
 'sold': None,
 'town': 'Pagani '}
{'city': '(PD)',
 'name': 'Sinclair ZX Spectrum 48k',
 'price': 280.0,
 'shipping': None,
 'sold': None,
 'town': 'Villafranca Padovana '}
{'city': '(SA)',
 'name': 'Computer sinclair zx spectrum 48k - testato 100%',
 'price': 150.0,
 'shipping': 'Spedizione disponibile',
 